In [1]:
%pip install pickleshare



  Using cached pickleshare-0.7.5-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached pickleshare-0.7.5-py2.py3-none-any.whl (6.9 kB)


In [2]:
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5
%pip install -qr requirements.txt 
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output 

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

d:\SENSEMI\yolov5


Cloning into 'yolov5'...
d:\SENSEMI\.venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Setup complete. Using torch 2.5.1+cpu (CPU)


In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="VW01wCar4Rx8GjuSHfGy")
project = rf.workspace("iris-annotation").project("irispupille")
version = project.version(1)
dataset = version.download("yolov5")
                
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to irispupille-1 in yolov5pytorch:: 100%|██████████| 912/912 [00:00<00:00, 3062.30it/s]


In [4]:
%cd "D:\\SENSEMI\\yolov5"

D:\SENSEMI\yolov5


In [5]:
%%capture output
!python train.py --img 640 --batch 16 --epochs 150 --data "D:\\SENSEMI\\yolov5\\irispupille-1\\data.yaml" --weights yolov5s.pt --cache

In [4]:
import os

path ="D:\\SENSEMI\\yolov5\\irispupille-1\\data.yaml"

if os.path.exists(path):
    print("The path exists.")
else:
    print("The path does not exist.")


The path exists.


In [6]:
!python detect.py --weights "D:\\SENSEMI\\yolov5\\runs\\train\\exp\\weights\\best.pt" --source "D:\\SENSEMI\\yolov5\\enhanced_eyes" --conf-thres 0.5


detect: weights=['D:\\\\SENSEMI\\\\yolov5\\\\runs\\\\train\\\\exp\\\\weights\\\\best.pt'], source=D:\\SENSEMI\\yolov5\\enhanced_eyes, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: detected dubious ownership in repository at 'D:/SENSEMI/yolov5'
'D:/SENSEMI/yolov5' is owned by:
	BUILTIN/Administrators (S-1-5-32-544)
but the current user is:
	AYUSH/HP (S-1-5-21-2724557807-3718659800-2024191519-1001)
To add an exception for this directory, call:

	git config --global --add safe.directory D:/SENSEMI/yolov5
YOLOv5  2024-12-15 Python-3.10.10 torch-2.5.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519

In [7]:
!python test.py --weights "D:\\SENSEMI\\yolov5\\runs\\train\\exp\\weights\\best.pt" --data "D:\\SENSEMI\\yolov5\\irispupille-1\\data.yaml" --img 640


Running custom YOLOv5 test script...


fatal: detected dubious ownership in repository at 'D:/SENSEMI/yolov5'
'D:/SENSEMI/yolov5' is owned by:
	BUILTIN/Administrators (S-1-5-32-544)
but the current user is:
	AYUSH/HP (S-1-5-21-2724557807-3718659800-2024191519-1001)
To add an exception for this directory, call:

	git config --global --add safe.directory D:/SENSEMI/yolov5
YOLOv5  2024-12-15 Python-3.10.10 torch-2.5.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

test: Scanning D:\SENSEMI\yolov5\irispupille-1\test\labels...:   0%|          | 0/45 [00:00<?, ?it/s]
test: Scanning D:\SENSEMI\yolov5\irispupille-1\test\labels... 1 images, 0 backgrounds, 0 corrupt:   2%|▏         | 1/45 [00:17<12:30, 17.05s/it]
test: Scanning D:\SENSEMI\yolov5\irispupille-1\test\labels... 45 images, 0 backgrounds, 0 corrupt: 100%|██████████| 45/45 [00:17<00:00,  2.63it/s]
test: New cache created: D:\SENSEMI\yolov5\irispupille-1\test\labels.cache

                 Class     Images  Instances      

In [9]:
import cv2
import os

# Load the eye detector
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Directory to store cropped eye images
output_dir = "detected_eyes"
os.makedirs(output_dir, exist_ok=True)

# Start video capture
cap = cv2.VideoCapture(0)

# Flags to track detected eyes
person_counter = 1  # Counter to store images for different people

while True:
    ret, img = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)

    for (ex, ey, ew, eh) in eyes:
        eye_center_x = ex + ew // 2
        frame_center_x = img.shape[1] // 2

        # Determine if it's the left or right eye based on position
        if eye_center_x < frame_center_x:
            eye_label = "Left Eye"
            color = (0, 255, 0)  # Green for left eye
        else:
            eye_label = "Right Eye"
            color = (255, 0, 0)  # Blue for right eye

        # Draw a rectangle around the detected eye
        cv2.rectangle(img, (ex, ey), (ex + ew, ey + eh), color, 2)
        cv2.putText(img, eye_label, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the frame with detected eyes
    cv2.imshow('Eye Detection', img)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('s'):  # Save the eyes when 's' is pressed
        for (ex, ey, ew, eh) in eyes:
            eye_center_x = ex + ew // 2
            frame_center_x = img.shape[1] // 2

            if eye_center_x < frame_center_x:
                eye_label = "Left Eye"
                eye_region = img[ey:ey + eh, ex:ex + ew]
                cv2.imwrite(os.path.join(output_dir, f"person_{person_counter}_left_eye.jpg"), eye_region)
            else:
                eye_label = "Right Eye"
                eye_region = img[ey:ey + eh, ex:ex + ew]
                cv2.imwrite(os.path.join(output_dir, f"person_{person_counter}_right_eye.jpg"), eye_region)

        print(f"Eyes detected and saved for person {person_counter}.")
        person_counter += 1

    elif key == ord('q'):  # Quit when 'q' is pressed
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


Eyes detected and saved for person 1.


In [1]:
import cv2
import os
import numpy as np

# Directory to store cropped eye images
input_dir = "C:\\Users\\HP\\OneDrive\\Desktop\\table"
output_dir = "enhanced_eyes"
os.makedirs(output_dir, exist_ok=True)

def remove_specular_reflection(image):
    """
    Removes specular reflections from the eye image using inpainting.
    """
    # Threshold to create a mask for bright spots (specular reflections)
    _, mask = cv2.threshold(image, 230, 255, cv2.THRESH_BINARY)  # Adjust threshold as needed
    
    # Dilate the mask to cover surrounding regions of reflections
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.dilate(mask, kernel, iterations=1)
    
    # Inpaint the regions covered by the mask
    inpainted_image = cv2.inpaint(image, mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)
    return inpainted_image

def enhance_image(image_path, output_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Step 1: Remove Specular Reflections
    no_reflection_image = remove_specular_reflection(image)

    # Step 2: Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    clahe_image = clahe.apply(no_reflection_image)

    # Step 3: Apply Bilateral Filtering for noise reduction while preserving edges
    bilateral_filtered_image = cv2.bilateralFilter(clahe_image, d=15, sigmaColor=100, sigmaSpace=100)

    # Step 4: Apply Sharpening
    sharpening_kernel = np.array([[0, -1, 0], [-1, 5.5, -1], [0, -1, 0]])
    sharpened_image = cv2.filter2D(bilateral_filtered_image, -1, sharpening_kernel)

    # Save the enhanced image
    cv2.imwrite(output_path, sharpened_image)

# Process each image in the detected_eyes directory
for filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, filename)

    # Skip if it's not an image file
    if not (filename.lower().endswith('.jpg') or filename.lower().endswith('.png')):
        continue

    # Generate output file path
    output_path = os.path.join(output_dir, filename)

    # Enhance the image
    enhance_image(input_path, output_path)
    print(f"Enhanced image saved to {output_path}")

print("All images enhanced and saved in the 'enhanced_eyes' directory.")


Enhanced image saved to enhanced_eyes\aL.jpg
Enhanced image saved to enhanced_eyes\aR.jpg
Enhanced image saved to enhanced_eyes\bL.jpg
Enhanced image saved to enhanced_eyes\bR.jpg
Enhanced image saved to enhanced_eyes\himL.jpg
Enhanced image saved to enhanced_eyes\himR.jpg
Enhanced image saved to enhanced_eyes\mL.jpg
Enhanced image saved to enhanced_eyes\mR.jpg
Enhanced image saved to enhanced_eyes\rL.jpg
Enhanced image saved to enhanced_eyes\rR.jpg
Enhanced image saved to enhanced_eyes\sL.jpg
Enhanced image saved to enhanced_eyes\sR.jpg
Enhanced image saved to enhanced_eyes\yL.jpg
Enhanced image saved to enhanced_eyes\yR.jpg
All images enhanced and saved in the 'enhanced_eyes' directory.


In [7]:
from ultralytics import YOLO
import cv2
import os

# Load your trained YOLOv5 model
model_path = "D:\\SENSEMI\\yolov5\\runs\\train\\exp\\weights\\best.pt"
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Paths for enhanced left and right eye images
enhanced_images_dir = "D:\\SENSEMI\\enhanced_eyes"
output_dir = "D:\\SENSEMI\\processed_eyes"
os.makedirs(output_dir, exist_ok=True)

# Class indices for iris and pupil (update according to your dataset)
IRIS_CLASS = 0  # Example: Class ID for iris
PUPIL_CLASS = 1  # Example: Class ID for pupil

# Function to process an image and calculate the ratio
def process_eye(image_path, eye_label):
    results = model.predict(source=image_path, conf=0.5, save=False)
    img = cv2.imread(image_path)

    iris_width = None
    pupil_width = None

    # Process detection results
    for result in results:
        for box, cls in zip(result.boxes.xyxy, result.boxes.cls):
            x_min, y_min, x_max, y_max = map(int, box)
            width = x_max - x_min

            # Check class and assign width
            if cls == IRIS_CLASS:
                iris_width = width
                color = (255, 0, 0)  # Blue for iris
            elif cls == PUPIL_CLASS:
                pupil_width = width
                color = (0, 255, 0)  # Green for pupil

            # Draw bounding box
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, 2)

    # Compute and display ratio in console
    ratio_text = f"{eye_label} Ratio: Not Detected"
    if iris_width and pupil_width:
        ratio = iris_width / pupil_width  # Changed to iris-to-pupil ratio
        ratio_text = f"{eye_label} IP Ratio: {ratio:.2f}"
        print(f"{os.path.basename(image_path)}: {ratio_text}")
    else:
        print(f"{os.path.basename(image_path)}: {ratio_text}")

    # Save the processed image with only bounding boxes
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, img)
    print(f"Processed image saved to: {output_path}")

# Process all enhanced images for left and right eyes
for filename in os.listdir(enhanced_images_dir):
    filepath = os.path.join(enhanced_images_dir, filename)
    if "left_eye" in filename.lower():
        process_eye(filepath, "Left Eye")
    elif "right_eye" in filename.lower():
        process_eye(filepath, "Right Eye")


In [2]:
#custom names
import torch
import cv2
import os

# Load the YOLOv5 model
model_path = "D:\\SENSEMI\\yolov5\\runs\\train\\exp\\weights\\best.pt"
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Input and output directories
enhanced_images_dir = "D:\\SENSEMI\\yolov5\\enhanced_eyes"  # Directory containing generic images
output_dir = "D:\\SENSEMI\\processed_eyes"  # Output directory for processed images
os.makedirs(output_dir, exist_ok=True)

# Class indices for iris and pupil
IRIS_CLASS = 0  # Class ID for iris
PUPIL_CLASS = 1  # Class ID for pupil

# Function to process an image and calculate the IP ratio
def process_image(image_path):
    results = model(image_path)  # YOLOv5 model inference
    img = cv2.imread(image_path)

    iris_width = None
    pupil_width = None

    # Process detection results
    for *box, conf, cls in results.xyxy[0]:  # Extract bounding boxes, confidence, and class
        x_min, y_min, x_max, y_max = map(int, box)
        width = x_max - x_min

        # Assign width based on class
        if cls == IRIS_CLASS:
            iris_width = width
            color = (255, 0, 0)  # Blue for iris
        elif cls == PUPIL_CLASS:
            pupil_width = width
            color = (0, 255, 0)  # Green for pupil

        # Draw bounding box on the image
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, 2)

    # Compute and log the IP ratio
    filename = os.path.basename(image_path)
    if iris_width and pupil_width:
        ratio = iris_width / pupil_width  # Iris-to-pupil ratio
        print(f"{filename}: IP Ratio = {ratio:.2f}")
    else:
        print(f"{filename}: IP Ratio = Not Detected")

    # Save the processed image with bounding boxes
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, img)

# Process all valid images in the directory
for filename in os.listdir(enhanced_images_dir):
    filepath = os.path.join(enhanced_images_dir, filename)

    # Ensure the file is a valid image format
    if filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
        process_image(filepath)
    else:
        print(f"Skipped non-image file: {filename}")


d:\SENSEMI\.venv\lib\site-packages\torch\hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\HP/.cache\torch\hub\master.zip
YOLOv5  2024-12-17 Python-3.10.10 torch-2.5.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
C:\Users\HP/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: Futur

aL.jpg: IP Ratio = 2.84


C:\Users\HP/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


aR.jpg: IP Ratio = 0.46


C:\Users\HP/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

sR.jpg: IP Ratio = Not Detected


C:\Users\HP/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


yL.jpg: IP Ratio = 3.70


C:\Users\HP/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


yR.jpg: IP Ratio = Not Detected


In [ ]:
from ultralytics import YOLO
model=YOLO("yolo11n.pt")
trained_results=model.train(data="D:\\mp\\data.yaml", epochs=100,imgsz=416,device="cpu")
metrices=model.val()


In [32]:
!yolo predict model="D:\\mp\\runs\\detect\\train\\weights\\best.pt" source="C:\\Users\\HP\\OneDrive\\Desktop\\pdd_L.jpg" conf=0.5


Ultralytics 8.3.31 🚀 Python-3.10.10 torch-2.5.1+cpu CPU (13th Gen Intel Core(TM) i5-1335U)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs

image 1/1 C:\Users\HP\OneDrive\Desktop\pdd_L.jpg: 320x416 1 iris, 3 pupils, 161.7ms
Speed: 4.0ms preprocess, 161.7ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 416)
Results saved to runs\detect\predict5
💡 Learn more at https://docs.ultralytics.com/modes/predict
